In [1]:
from src.component.data_info import *
from src.paths import *
import pandas as pd

In [2]:
HOPSWORKS_PROJECT_NAME = 'electricity_demand2'

In [3]:
#datetime.now().year


In [4]:
datetime.now()

datetime.datetime(2025, 2, 3, 13, 48, 34, 356055)

In [5]:
import os
from dotenv import load_dotenv
from src.paths import PARENT_DIR

# load key-value pairs from .env file located in the parent directory
load_dotenv(PARENT_DIR / '.env')

HOPSWORKS_API_KEY = os.environ['HOPSWORKS_API_KEY']

In [6]:
start=datetime(2023,1,1)
end=datetime(2025,1,1)
full_data=load_full_data(start, end)

Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-01.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-02.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-03.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-04.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-05.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-06.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-07.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-08.json
Loading file D:\Electricity_demand_predictor2\data\raw\electricity_raw_data\hourly_demand_2023-01-09.json
Loading file D:\Electricity_demand_predictor2\

In [7]:
full_data

,date,sub_region_code,demand,temperature_2m
0,2023-01-02 00:00:00+00:00,0,1656,10.8585
1,2023-01-02 00:00:00+00:00,1,1132,10.8585
2,2023-01-02 00:00:00+00:00,2,1858,10.8585
3,2023-01-02 00:00:00+00:00,3,715,10.8585
4,2023-01-02 00:00:00+00:00,4,919,10.8585
...,...,...,...,...
201207,2025-01-01 01:00:00+00:00,6,1174,2.1085
201208,2025-01-01 01:00:00+00:00,7,325,2.1085
201209,2025-01-01 01:00:00+00:00,8,695,2.1085
201210,2025-01-01 01:00:00+00:00,9,5601,2.1085


In [8]:
# string to datetime
full_data['date'] = pd.to_datetime(full_data['date'], utc=True)

# add column with Unix epoch milliseconds
full_data['seconds'] = full_data['date'].astype('int64') // 10**6

In [9]:
full_data['date'] .max()

Timestamp('2025-01-02 00:00:00+0000', tz='UTC')

In [10]:
full_data.sort_values(by=['sub_region_code'], inplace=True)
full_data.isna().sum()

date               0
sub_region_code    0
demand             0
temperature_2m     0
seconds            0
dtype: int64

In [11]:
#full_data['sub_region_code'] = full_data['sub_region_code'].astype('int64')

In [12]:
import hopsworks

d:\poetry_virtualenvs\src-o1faXYQI-py3.12\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
project = hopsworks.login(
    project=HOPSWORKS_PROJECT_NAME,
    api_key_value=HOPSWORKS_API_KEY
)

2025-02-03 13:48:48,143 INFO: Initializing external client
2025-02-03 13:48:48,145 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-02-03 13:48:49,153 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1212591


In [14]:
feature_store = project.get_feature_store()

In [15]:
FEATURE_GROUP_NAME = 'electricity_demand_feature_group2'
FEATURE_GROUP_VERSION = 3

In [16]:
feature_group = feature_store.get_or_create_feature_group(
    name=FEATURE_GROUP_NAME,
    version=FEATURE_GROUP_VERSION,
    description="Time-series data at hourly frequency",
    primary_key = ['sub_region_code', 'date'],
    event_time='date',
)

In [17]:
feature_group.insert(full_data, write_options={"wait_for_job": False})

Uploading Dataframe: 100.00% |██████████| Rows 201212/201212 | Elapsed Time: 00:12 | Remaining Time: 00:00
Use fg.materialization_job.run(args=-op offline_fg_materialization -path hdfs:///Projects/electricity_demand2/Resources/jobs/electricity_demand_feature_group2_3_offline_fg_materialization/config_1738542280040) to trigger the materialization job again.


(Job('electricity_demand_feature_group2_3_offline_fg_materialization', 'SPARK'),
 None)